### Object Representations:

Standard way to get a string representation from any objectL:
    
`repr`: return a string repping object as developer wants to see it (what we get in console or debugger)
- `__repr__`

`str`: return a string repping object as user wants to see it. This is the return when we `print()` an object
- `__str__`

Alternative reps:

- `__bytes__`: called by `bytes()` to get byte sequence

- `__format__`: used by `f-strings`, getting string displays using special formatting codes

In [1]:
# source: https://github.com/fluentpython/example-code-2e/blob/master/11-pythonic-obj/vector2d_v0.py
from array import array
import math


class Vector2d:
    typecode = 'd'  # used in __bytes__

    def __init__(self, x, y):
        self.x = float(x)    # 
        self.y = float(y)

    def __iter__(self):
        """Using a generator expression for unpacking"""
        return (i for i in (self.x, self.y))  # makes it iterable

    def __repr__(self):
        """*self is an iterable due to above, so feeds x & y"""
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)  # <4>

    def __str__(self):
        return str(tuple(self))  # <5>

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +  # <6>
                bytes(array(self.typecode, self)))  # <7>

    def __eq__(self, other):
        return tuple(self) == tuple(other)  # let's us compare vals

    def __abs__(self):
        return math.hypot(self.x, self.y)  # magnitude

    def __bool__(self):
        return bool(abs(self))  # a 0 would be false, otherwise true

In [4]:
# testing
v1 = Vector2d(3,4)

print(v1) # __str__ output

# repr
v1

(3.0, 4.0)


Vector2d(3.0, 4.0)

In [15]:
# eval just evaluates expression
# if legal statement, it will be executed
x = 5
eval("print(x)") # actual statement we can execute
eval("x") # same as executing x

5


5

In [17]:
# proves repr is same as constructor call (not sure I follow)
v1_clone = eval(repr(v1))
v1_clone == v1

True

In [18]:
bytes(v1) # binary representation

b'd\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x10@'

In [23]:
# equal is interesting. 
v1 = Vector2d(3,4)
v2 = Vector2d(3,4)
print(v1 == v2)

# also works for a totally different object, like a list
print(v1 == [3,4])

True
True


### Classmethod vs staticmethod: 

`classmethod`:
- operate on class and not on instances.
- receives class as argument, most common use is for alternative constructions. 
    - used for ways to create an object with a different signature to `__init__`

`staticmethod`:
- knows nothing about the class or instance it was called on
- could use the a standard function, but can be helpful to package up within a class if it is a helpful function (avoid pollution or confusion from lots of "free functions"

In [24]:
class A(object):
    def foo(self, x):
        print(f"executing foo({self}, {x})")

    @classmethod
    def class_foo(cls, x):
        print(f"executing class_foo({cls}, {x})")

    @staticmethod
    def static_foo(x):
        print(f"executing static_foo({x})")

a = A()

In [26]:
a.foo(1) # typical way an object calls a method

executing foo(<__main__.A object at 0x1143c9f70>, 1)


In [28]:
A.class_foo(1) == a.class_foo(1)

executing class_foo(<class '__main__.A'>, 1)
executing class_foo(<class '__main__.A'>, 1)


True

In [30]:
a.static_foo(1) # behavior like plain functions, but we call from class
A.static_foo("hello")

executing static_foo(1)
executing static_foo(hello)


### classmethod: more examples

Good resource: https://stackabuse.com/pythons-classmethod-and-staticmethod-explained/

In the example we want to generate various formats for a class to be instantiated from

In [33]:
class Student(object):

    def __init__(self, first_name, last_name):
        self.first_name = first_name
        self.last_name = last_name

scott = Student('Scott',  'Robinson')
print(scott.first_name, scott.last_name)

Scott Robinson


In [38]:
class Student(object):

    def __init__(self, first_name, last_name):
        self.first_name = first_name
        self.last_name = last_name
        
    @classmethod
    def from_string(cls, name_str):
        first_name, last_name = map(str, name_str.split(' '))
        student = cls(first_name, last_name)
        return student
    
# typical approach
scott1 = Student('Scott',  'Robinson')
print(scott1.first_name, scott1.last_name)

# parsing from a string, a different approach
scott2 = Student.from_string('Scott Robinson')
print(scott2.first_name, scott2.last_name)

Scott Robinson
Scott Robinson


In [43]:
# Maybe a more relevant example for DS work:
# We could create an instance with a list or we could pass in
# a csv (comma-separate string in this example)
class ClassGrades:

    def __init__(self, grades):
        self.grades = grades

    @classmethod
    def from_csv(cls, grade_csv_str):
        # this is a bit clunky.
        grades = list(map(int, grade_csv_str.split(', ')))

        return cls(grades)
# list
class_grades_valid = ClassGrades([90,80,85,94,70])
print(class_grades_valid.grades)

# "csv"
class_grades_valid = ClassGrades.from_csv('90, 80, 85, 94, 70')
print(class_grades_valid.grades)

[90, 80, 85, 94, 70]
[90, 80, 85, 94, 70]


### Formatted Displays